In [1]:
from io import open
from __future__ import unicode_literals, print_function, division
import random
import unicodedata
import string
import pandas as pd
import re
import torch
import numpy
import torch.nn as nn
from torch import optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab , build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [2]:
%%capture
!pip install wandb

In [3]:
import wandb

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


In [5]:
print(device)

cpu


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
!unzip -qq "/content/drive/My Drive/Colab Notebooks/aksharantar_sampled.zip" -d "/content"

In [9]:
class Lang:
    def __init__(self, name):
        self.char2index = {'#': 0, '$': 1, '^': 2}
        self.name = name
        self.n_chars = 3  # Count
        self.char2count = {'#': 1, '$': 1, '^': 1}
        self.data = {}
        self.index2char = {0: '#', 1: '$', 2: '^'}    
        

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char in self.char2index:
            self.char2count[char] += 1
        else:
            self.char2index[char] = self.n_chars
            self.index2char[self.n_chars] = char
            self.n_chars += 1
            self.char2count[char] = 1

In [10]:
# return max length of input and output words
def maxLength(data):
    size=len(data)
    ip_mlen, op_mlen = 0, 0

    for i in range(size):
        input = data[0][i]
        output = data[1][i]
        x= len(input)
        if(x>ip_mlen):
            ip_mlen=x
        y= len(output)
        if(y>op_mlen):
            op_mlen=y

    return ip_mlen, op_mlen

In [40]:
def preprocess(data, input_lang, output_lang):
    x= len(data)
    maxlenInput, maxlenOutput = maxLength(data)
    maxlenInput = 25
    input = numpy.zeros((x, maxlenInput ))
    output = numpy.zeros((x, maxlenOutput + 2))
    # maxlenInput, maxlenOutput = maxLength(data)
    unknown = input_lang.char2index['$']

    for i in range(x):   
        b=maxlenInput + 1
        ip = data[0][i][:maxlenInput] 
        # ip = data[0][i].ljust(b, '#')
        c=maxlenOutput + 2
        op = '^' + data[1][i]
        op = op.ljust(c, '#')
        
        b=enumerate(ip)
        for index, char in b:
            r= input_lang.char2index[char]
            if input_lang.char2index.get(char) is not None:
                input[i][index] = r
            else:
                input[i][index] = unknown
        

        c=enumerate(op)
        for index, char in c:
            s= output_lang.char2index[char]
            if output_lang.char2index.get(char) is not None:
                output[i][index] = s
            else:
                output[i][index] = unknown  

    print(input.shape)
    inp=torch.from_numpy(input)
    print(output.shape)
    
    out= torch.from_numpy(output)
    return TensorDataset(inp,out )


In [41]:
def loadData(lang):
    input_lang = Lang('eng')
    train_df = pd.read_csv(f"/content/aksharantar_sampled/{lang}/{lang}_train.csv", header = None)
    output_lang = Lang(lang)
    
    # add the words to the respective languages
    size=len(train_df)
    for i in range(size):
        x=train_df[0][i]
        input_lang.addWord(x)
        y=train_df[1][i]
        output_lang.addWord(y)

    
    trainDataset = preprocess(train_df, input_lang, output_lang)
    val_df = pd.read_csv(f"/content/aksharantar_sampled/{lang}/{lang}_valid.csv", header = None)
    valDataset = preprocess(val_df, input_lang, output_lang)
    test_df = pd.read_csv(f"/content/aksharantar_sampled/{lang}/{lang}_test.csv", header = None)
    target_characters = set()
    testDataset = preprocess(test_df, input_lang, output_lang)
    
    return trainDataset, testDataset, valDataset, input_lang, output_lang
    
trainData, testData, valData, ipLang, opLang = loadData('tel')

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


(51200, 25)
(51200, 23)


/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


(4096, 25)
(4096, 21)
(4096, 25)
(4096, 22)


In [19]:
len(trainData)

51200

In [20]:
wandb.login(key ="ab8429eef239aaf26799fd710e5ea6aec30ab090")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
sweep_config = {
  'name':"my-sweep",
  'method': 'bayes',
  'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
  'parameters': {
        'epochs':{
            'values': [10, 15, 20]
        },
        'learn_rate': {
            'values': [0.00001, 0.0001]
        },
        'cell_type':{
            'values':["RNN", "LSTM", "GRU"]
        },
        'num_decoders':{
            'values': [1,2]
        },
        'embedding_dim': {
            'values': [64, 128, 256]
        },
        'bidirectional':{
            'values' : ["Yes","No"]
        },
        'batch_size':{
            'values':[ 64, 128, 256]
        },
        'hidden_dim':{
            'values':[64, 128, 256]
        },
        'attention':{
            'values': ["Yes","No"]
        },
        'teach_ratio':{
            'values':[0.5]
        },
        'num_encoders':{
            'values': [1]
        },
        'dropout':{
            'values':[0, 0.1, 0.2]
        }
                  
    }
}

config_defaults={
    'num_decoders': 3,
    'learn_rate' : 0.0001,
    'bidirectional': 'Yes',
    'embedding_dim': 32,
    'teach_ratio': 0.5,
    'cell_type': "LSTM",
    'hidden_dim' : 128,
    'num_encoders': 2,
    'attention': "Yes",
    'epochs': 10,
    'batch_size': 16,
    'attention': "Yes",
    'dropout': 0.2
    
}

In [22]:
class MyEncoder(nn.Module):
    # MyEncoder class represents the encoder component of a sequence-to-sequence model.
    def __init__(self, input_dim, hidden_dim, embedding_dim, 
                 num_layers, cell_type,
                  bidirectional, dropout, batch_size) :
        # Initialize variables
        super(MyEncoder, self).__init__()
        x= hidden_dim
        self.embedding = nn.Embedding(num_embeddings=input_dim,embedding_dim= embedding_dim)
        self.hidden_dim = x
        y= num_layers
        self.num_layers = y
        self.bidirectional = True if bidirectional == 'Yes' else False
        p = embedding_dim
        self.embedding_dim= p
        n= cell_type
        self.cell_type = n
        m= batch_size
        self.batch_size = m
        self.dropout = nn.Dropout(dropout)
        if self.cell_type == "GRU":
            gr= nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=self.bidirectional, dropout=dropout)
            self.rnn = gr
        elif self.cell_type == "LSTM":
            ls=nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=self.bidirectional, dropout=dropout)
            self.rnn = ls
        elif self.cell_type == "RNN":
            rn=nn.RNN(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=self.bidirectional, dropout=dropout)
            self.rnn = rn
    # Initializing the initial hidden state for the encoder
    def initialize_hidden(self):
        num_directions = 2 if self.bidirectional else 1
        if self.cell_type != "LSTM":
            return torch.zeros(self.num_layers * num_directions, self.batch_size, self.hidden_dim, device=device)
        else:
            val=self.num_layers * num_directions
            return (torch.zeros(val, self.batch_size, self.hidden_dim, device=device),
                    torch.zeros(val, self.batch_size, self.hidden_dim, device=device))
            
    # Perform the forward pass of the encoder
    def forward(self, input, hidden): # input shape (seq_len, batch_size) hidden shape tuple for lstm, otherwise single
        output = self.dropout(self.embedding(input.long()).view(-1,self.batch_size, self.embedding_dim)) # output shape (seq_len, batch_size, embedding size)

        output, hidden = self.rnn(output, hidden) # for LSTM hidden is a tuple
        if self.bidirectional:
            if self.cell_type != "LSTM":
                hidden=hidden.resize(2,self.num_layers,self.batch_size,self.hidden_dim)
                vc=torch.add(hidden[0],hidden[1])
                hidden=vc/2
            else:
                hidden_state = hidden[0].resize(2,self.num_layers,self.batch_size,self.hidden_dim)
                ap=torch.add(hidden_state[0],hidden_state[1])
                cell_state = hidden[1].resize(2,self.num_layers,self.batch_size,self.hidden_dim)
                tg=torch.add(cell_state[0],cell_state[1])
                hidden = (ap/2,tg/2)

            split_tensor= torch.split(output, self.hidden_dim, dim=-1)
            output=torch.add(split_tensor[0],split_tensor[1])/2  
        return output, hidden

In [23]:
class MyDecoder(nn.Module):
    def __init__(self, hidden_dim, output_size, embedding_dim, num_layers, 
                 cell_type, dropout, batch_size):
        # Initialize variables
        super(MyDecoder, self).__init__()
        m= hidden_dim
        self.embedding = nn.Embedding(output_size, embedding_dim)
        self.hidden_dim = m
        n= num_layers
        self.num_layers = n
        p= batch_size
        self.batch_size = p
        q=embedding_dim
        self.embedding_dim=q
        o= cell_type.lower()
        self.cell_type = o
        
        if self.cell_type == "gru":
            gr= nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
            self.rnn = gr
        elif self.cell_type == "lstm":
            ls= nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
            self.rnn = ls
        else:
            rn=nn.RNN(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
            self.rnn = rn

        r= nn.Linear(hidden_dim, output_size)
        self.out = r 
        s= nn.LogSoftmax(dim=2)
        self.softmax = s
    # Perform the forward pass of the decoder
    def forward(self, input, hidden):
        output = F.relu(self.embedding(input.long()).view(-1, self.batch_size, self.embedding_dim))
        output, hidden = self.rnn(output, hidden)
        val=self.out(output)
        output = self.softmax(val)
        return output, hidden


In [31]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_dim, output_size, embedding_dim, num_layers,
                 cell_type, dropout, batch_size, max_length):
        
        super(AttentionDecoderRNN, self).__init__()
        hd=hidden_dim
        self.hidden_dim = hd
        nl=num_layers
        self.num_layers = nl
        ct=cell_type
        self.cell_type = ct
        bs= batch_size
        self.batch_size =bs
        ed=embedding_dim
        self.embedding_dim = ed
        self.embedding = nn.Embedding(output_size, embedding_dim)
        ml= max_length
        self.max_length =ml
        self.attention = nn.Linear(hidden_dim + embedding_dim, self.max_length)
        dp=dropout
        self.dropout = dp
        self.dropout = nn.Dropout(self.dropout)
        he=hidden_dim + embedding_dim
        self.attention_combine = nn.Linear(he, hidden_dim)
        self.softmax = nn.LogSoftmax(dim=2)
        if self.cell_type == "GRU":
            gr=nn.GRU(hidden_dim, hidden_dim, num_layers=num_layers)
            self.rnn = gr
        elif self.cell_type == "LSTM":
            ls=nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers)
            self.rnn = ls
        else:
            rn=nn.RNN(hidden_dim, hidden_dim, num_layers=num_layers)
            self.rnn = rn

        self.out = nn.Linear(hidden_dim, output_size)
        

    def forward(self, input, hidden, encoder_outputs): #input shape (1, batch_size) 
        # embedded shape (1, batch_size, embedding_dim)
        embedded = F.relu(self.embedding(input.long()).view(-1, self.batch_size, self.embedding_dim))

        # Compute attention scores
        if self.cell_type == "LSTM":
            attn_hidden = torch.mean(hidden[0], dim=0)
        else:
            attn_hidden = torch.mean(hidden, dim = 0)
        vas=torch.cat((embedded, attn_hidden.unsqueeze(0)), dim=2)
        attn_scores = self.attention(vas) # attn_scores shape (1, batch_size, max_length)
        et=encoder_outputs.transpose(0, 1)
        attn_weights = F.softmax(attn_scores, dim=0)  # attn_scores shape (1, 16, 25)
        at=attn_weights.transpose(0, 1)

        # Apply attention weights to encoder outputs
        attn_applied = torch.bmm(at,et )
        
        # Combine attention output and embedded input
        combined = torch.cat((embedded, attn_applied.transpose(0, 1)), dim=2)
        combined = self.attention_combine(combined)
        combined = F.relu(combined) # shape (1, batch_size, hidden_dim)

        # Run through the RNN
        output, hidden = self.rnn(combined, hidden)
        # output shape: (1, batch_size, hidden_dim)

        # Pass through linear layer and softmax activation
        output = self.out(output)  # shape: (1, batch_size, output_size)
        output = self.softmax(output)

        return output, hidden, attn_weights


In [25]:
def calculate_exact_matches(pred, target):
    """
    Counts the number of exact matches between the rows of the preds tensor and the y tensor.
    pred: tensor of shape (batch_size, seq_len)
            Tensor containing predicted values.
    target: tensor of shape (batch_size, seq_len)
            Tensor containing target values.
    Returns:Number of rows in the preds tensor that match exactly with each row in the target tensor.
    """
    len1=pred.shape[0]
    count=0;
    for i in range(len1):
      len2=pred.shape[1]
      flag = True
      for j in range(len2):
        if(target[i][j]!=pred[i][j]):
          flag=False
          break;
      x=1
      if(flag):
        count+=x;
    
    return count


In [26]:
# Evaluate the performance of the encoder-decoder model on the given data.
def evaluate_model(data,encoder, decoder,output_size,batch_size,hidden_dim,num_encoders,num_decoders, cell_type,attention):
    
    loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    running_loss = 0
    correct =0
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    total=0

    seq_len = 0
    with torch.no_grad():
      for j,(x,y) in enumerate(loader):
        loss=0
        encoder.eval()
        val= num_decoders - num_encoders
        decoder.eval()

        x = x.to(device)
        x = x.T

        y = y.to(device)
        y = y.T
        seq_len = len(y)
        
        encoder_hidden=encoder.initialize_hidden()
        decoder_input =y[0]
        encoder_output,encoder_hidden = encoder(x,encoder_hidden)
        
        # Handle different numbers of layers in the encoder and decoder
        if num_encoders == num_decoders:
            decoder_hidden = encoder_hidden 
        else:
            if num_encoders > num_decoders:
                # Slice the hidden states of the encoder to match the decoder layers
                if cell_type != "LSTM":
                    decoder_hidden = encoder_hidden[-num_decoders:]
                else :
                    decoder_hidden = (encoder_hidden[0][-num_decoders:], encoder_hidden[1][-num_decoders:])
            else:
                
                # Copy all encoder hidden layers and then repeat the top layer
                if cell_type != "LSTM":
                    remaining_layers = val
                    top_layer_hidden = encoder_hidden[-1].unsqueeze(0) #top_layer_hidden shape (1, batch_size, hidden_dim)
                    extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
                    dm=0
                    decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=dm) 
                else:  
                    top_layer_hidden_h = encoder_hidden[0][-1].unsqueeze(0)
                    top_layer_hidden_c = encoder_hidden[1][-1].unsqueeze(0)
                    top_layer_hidden = (top_layer_hidden_h, top_layer_hidden_c)
                    remaining_layers = val
                    extra_hidden = (top_layer_hidden[0].repeat(remaining_layers, 1, 1), top_layer_hidden[1].repeat(remaining_layers, 1, 1))
                    dm=0
                    decoder_hidden = (torch.cat((encoder_hidden[0], extra_hidden[0]), dim=dm), torch.cat((encoder_hidden[1], extra_hidden[1]), dim=dm))

        sm=len(y)
        pred=torch.zeros(sm-1, batch_size)

        for k in range(1,sm):
          if attention == "Yes":
              decoder_output, decoder_hidden, atten_weights = decoder(decoder_input, decoder_hidden, encoder_output)
          else:
              decoder_output, decoder_hidden= decoder(decoder_input, decoder_hidden)
          lp=y[k].long()
          max_prob, index = decoder_output.topk(1) # max_prob shape (1, batch_size, 1)
          decoder_output = torch.squeeze(decoder_output)
          ind =k-1
          loss += loss_fun(decoder_output,lp)
          pred[ind]= torch.squeeze(index)
          decoder_input = index

        running_loss += loss.item()
        rs=y[1:,:].T
        correct += calculate_exact_matches(pred.T,rs)

    si= len(data) 
    den = (si* seq_len)    
    avg_loss = running_loss / den 
    #print("correct =", correct)
    num=correct / (si)
    avg_acc = 100 *num 
    return avg_loss, avg_acc

In [30]:

def train(sweeps = True, test = False):
    
    if sweeps == True: 
        wandb.init(config= config_defaults, project='DL_assign_3')   # if not test then run wandb sweeps
        configs=wandb.config    
    else:
        configs = config_defaults  # use the default configuration which has the best hyperparameters
    nd= configs['num_decoders']  
    num_decoders = nd
    lr=configs['learn_rate']
    learn_rate = lr
    bd=configs['bidirectional']
    bidirectional = bd
    ed=configs['embedding_dim']
    embedding_dim = ed
    tr=configs['teach_ratio']
    teach_ratio =tr 
    ct=configs['cell_type']
    cell_type = ct
    hd=configs['hidden_dim']
    hidden_dim =hd
    ne= configs['num_encoders']
    num_encoders = ne
    ep=configs['epochs']
    epochs = ep
    bs= configs['batch_size']
    batch_size =bs
    attention = configs['attention']
    dropout = configs['dropout']
    print(f"batch size {batch_size}, hidden size {hidden_dim}")

    input_len = ipLang.n_chars
    if sweeps:
       wandb.run.name='hidden_'+str(hidden_dim)+'_batch_'+str(batch_size)+'_embed_size_'+str(embedding_dim)+'_dropout_'+str(dropout)+'_cell_'+str(cell_type)
    
    encoder = MyEncoder(input_len, hidden_dim, embedding_dim, 
                 num_encoders, cell_type,
                  bidirectional, dropout, batch_size)
    output_len = opLang.n_chars
    if attention =="Yes":
        decoder = AttentionDecoderRNN(hidden_dim, output_len, embedding_dim, num_decoders, 
                 cell_type, dropout, batch_size, 25)
    else:
        decoder = MyDecoder(hidden_dim, output_len, embedding_dim, num_decoders, 
                 cell_type, dropout, batch_size)#dropout not used
    
    encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
    train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True)
    
    loss_fun=nn.CrossEntropyLoss(reduction="sum")    
    encoder.to(device)
    seq_len = 0
    decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
    val_loader = DataLoader(valData, batch_size=batch_size, shuffle=True)
    decoder.to(device)

    # Initialize variables for early stopping
    best_val_loss = float('inf')
    patience = 5
    epochs_without_improvement = 0
    
    for i in range(epochs):
        
        running_loss = 0.0
        train_correct = 0

        encoder.train()
        bq= enumerate(train_loader)
        decoder.train()

        for j,(train_x,train_y) in bq:
            
            val=num_decoders - num_encoders
            train_x = train_x.to(device)
            train_x=train_x.T
            encoder_optimizer.zero_grad()
            train_y = train_y.to(device)
            train_y=train_y.T
            seq_len = len(train_y)
            # lets move to the decoder
            decoder_input = train_y[0]

            decoder_optimizer.zero_grad()
            
            encoder_hidden=encoder.initialize_hidden()
            encoder_output,encoder_hidden = encoder(train_x,encoder_hidden)
            
            
            # Handle different numbers of layers in the encoder and decoder
            if num_encoders == num_decoders:
                decoder_hidden = encoder_hidden
                
            else:
                if num_encoders > num_decoders:
                    # Slice the hidden states of the encoder to match the decoder layers
                    if cell_type != "LSTM":
                        decoder_hidden = encoder_hidden[-num_decoders:]
                    else :
                        decoder_hidden = (encoder_hidden[0][-num_decoders:], encoder_hidden[1][-num_decoders:])
                else:
                    
                    # Copy all encoder hidden layers and then repeat the top layer
                    if cell_type != "LSTM":
                        remaining_layers = val
                        top_layer_hidden = encoder_hidden[-1].unsqueeze(0) #top_layer_hidden shape (1, batch_size, hidden_dim)
                        extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
                        dm=0
                        decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=dm)
                    else:
                        top_layer_hidden_h = encoder_hidden[0][-1].unsqueeze(0)
                        top_layer_hidden_c = encoder_hidden[1][-1].unsqueeze(0)
                        top_layer_hidden = (top_layer_hidden_h, top_layer_hidden_c)
                        remaining_layers = val
                        extra_hidden = (top_layer_hidden[0].repeat(remaining_layers, 1, 1), top_layer_hidden[1].repeat(remaining_layers, 1, 1))
                        dm=0
                        decoder_hidden = (torch.cat((encoder_hidden[0], extra_hidden[0]), dim=dm), torch.cat((encoder_hidden[1], extra_hidden[1]), dim=dm))

            loss = 0
            sg=len(train_y)-1
            correct = 0
           
            for k in range(0,sg ):
                
                if attention == "Yes":
                    decoder_output, decoder_hidden, atten_weights = decoder(decoder_input, decoder_hidden, encoder_output)
                else:
                    decoder_output, decoder_hidden= decoder(decoder_input, decoder_hidden) # decoder_output shape (1, batch_size, output_size)

                pq=torch.squeeze(decoder_output)
                max_prob, index = decoder_output.topk(1) # max_prob shape (1, batch_size, 1)
                rs= train_y[k+1].long()
                index = torch.squeeze(index) # shape (batch_size)
                # Apply teacher forcing
                use_teacher_forcing = True if random.random() < teach_ratio else False
                decoder_output = pq
                loss += loss_fun(decoder_output,rs)
                
                correct += (index == train_y[k+1]).sum().item()

                
                if use_teacher_forcing:
                    decoder_input = train_y[k+1]
                
                else:
                    decoder_input = index

            train_correct =train_correct + correct
            running_loss =running_loss+ loss.item()
            decoder_optimizer.step()
            loss.backward()
            encoder_optimizer.step()
            
        # find train loss and accuracy and print + log to wandb
        _, train_accuracy = evaluate_model(trainData,encoder, decoder,output_len,batch_size,hidden_dim,num_encoders,num_decoders, cell_type, attention)
        train_loss=running_loss/(len(trainData)* seq_len)
        print("After ",i+1,"epoch:")
        print("training loss:",train_loss)
        print("training accuracyy:",train_accuracy)
        if sweeps:
            wandb.log({"epoch": i, "train_loss": train_loss, "train_accuracy": train_accuracy})
        
        # find validation loss and accuracy and print + log to wandb
        val_loss, val_accuracy = evaluate_model(valData,encoder, decoder,output_len,batch_size,hidden_dim,num_encoders,num_decoders, cell_type, attention)
        print("validation loss:",val_loss)
        print("validation accuracy:",val_accuracy)
        if sweeps:
            wandb.log({"val_loss": val_loss, "val_accuracy": val_accuracy})
        
        # Check for early stopping
        if val_loss > best_val_loss:
            epochs_without_improvement += 1
            patience=5
            if epochs_without_improvement >= patience:
                print("Early stopping triggered. No improvement in validation loss.")
                break 
        else:
            epochs_without_improvement = 0
            best_val_loss = val_loss
            # Save the model weights
            torch.save(encoder.state_dict(), 'best_encoder.pt')
            torch.save(decoder.state_dict(), 'best_decoder.pt')
    # if testing mode is on print the test accuracy 
    if test:
        # Load the best model weights
        encoder.load_state_dict(torch.load('best_encoder.pt'))
        decoder.load_state_dict(torch.load('best_decoder.pt'))
        _, test_accuracy = evaluate_model(testData,encoder, decoder,output_len,batch_size,hidden_dim,num_encoders,num_decoders, bidirectional, cell_type)
        print(f"test accuracy {test_accuracy}")

In [43]:
train(sweeps = False, test = True)

In [45]:
# sweep_id=wandb.sweep(sweep_config, project="Assignment_3")
# wandb.agent(sweep_id,function=train)
wandb.agent(sweep_id= "3rd4g4gq",function=train, project="Assignment_3")

##Testing the Best Model on Test Data

In [46]:
train(test = True)

In [47]:
class AttnDecoder(nn.Module):
    def __init__(self,output_size,hidden_dim,embedding_dim,decoder_layers,batch_size,cell_type,dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        x=hidden_dim
        self.hidden_dim = x
        y=output_size
        self.output_size = y
        r=dropout_p
        self.dropout_p = r
        s=batch_size
        self.batch_size=s 
        t=cell_type
        self.U=nn.Linear(self.hidden_dim,self.hidden_dim,bias=False).to(device)
        self.cell_type=t
        
        u=embedding_dim
        self.embedding_dim= u
        self.softmax=nn.LogSoftmax()
        v=decoder_layers
        self.W=nn.Linear(self.hidden_dim,self.hidden_dim,bias=False).to(device)
        self.decoder_layers=v
        
        l=nn.Embedding(self.output_size, self.embedding_dim)
        self.embedding = l
        p= nn.Dropout(self.dropout_p)
        self.V=nn.Linear(self.hidden_dim,1,bias=False).to(device)
        self.dropout = p
        
        self.linear=nn.Linear(self.hidden_dim,output_size,bias=True)
        self.softmax1=nn.LogSoftmax(dim=2)
        if(cell_type=="GRU"):
            gr=nn.GRU(self.embedding_dim+self.hidden_dim, self.hidden_dim,self.decoder_layers,dropout = dropout_p)
            self.gru = gr
        if(cell_type=="LSTM"):
            ls=nn.LSTM(self.embedding_dim+self.hidden_dim, self.hidden_dim,self.decoder_layers,dropout = dropout_p)
            self.lstm = ls
        if(cell_type=="RNN"):
            rn=nn.RNN(self.embedding_dim+self.hidden_dim, self.hidden_dim,self.decoder_layers,dropout = dropout_p)
            self.rnn = rn

    def forward(self, input, hidden,encoder_outputs,word_length,state=None):
        T=word_length
        embedded = self.embedding(input).view(-1,self.batch_size, self.embedding_dim)
#         embedded = self.dropout(embedded)
        c=torch.zeros(self.batch_size,self.hidden_dim).to(device)
        temp=self.W(hidden[-1])
        
        for j in range(0,T):
            mk=(encoder_outputs[j])+temp
            e_j=self.V(torch.tanh(self.U(mk)))
            c+=self.softmax(e_j)*encoder_outputs[j]
        

        final_input=torch.cat((embedded,c.unsqueeze(0)),dim=2)
        
        
        if(self.cell_type=="GRU"):
            gr=self.gru(final_input,hidden)
            output,hidden=gr
        if(self.cell_type=="RNN"):
            rn=self.rnn(final_input,hidden)
            output,hidden=rn
        if(self.cell_type=="LSTM"):
            ls=self.lstm(final_input,(hidden,state))
            output, (hidden,state) =ls
        
        output1=self.softmax1(self.linear(output))
        
        if(self.cell_type=="GRU" or self.cell_type=="RNN"):
            return output1, hidden, c
        if(self.cell_type=="LSTM"):
            return output1, hidden, state, c
